In [ ]:
import os
import papermill as pm

# Training and validation data
train_dir = "../dataset/train-val-independent/train/"
validation_dir = "../dataset/train-val-independent/validation/"

epoch_num = 50

# Create "results" folder
os.makedirs("results_indep", exist_ok=True)

# Notebook name
input_notebook = "train_AST_fixed1_from_SSL.ipynb"

# Output file template
output_notebook_template = "results_indep/train_ASTfromSSL_result_seed{}_{}_{}.ipynb"

# Settings to test
optimizers = ["adamw"]
schedulers = ["cosine", "constant_with_warmup"]

# List of random seeds
seeds = [42, 123, 2024, 314, 777]

# Optimizer-specific learning rates
learning_rates = {
    "adamw": 5e-5,
    "rmsprop": 5e-4
}

# Initialize log file
log_file = "experiment_log_indep_25_02_24.txt"
with open(log_file, "a") as log:
    log.write("Experiment Log:\n\n")

# Papermill runs
results = []

# Run for every combination
for seed in seeds:
    for optimizer in optimizers:
        for scheduler in schedulers:
            output_notebook = output_notebook_template.format(seed, optimizer, scheduler)
            lr = learning_rates[optimizer]  # Selected LR

            print(f"Running experiment with seed={seed}, {optimizer}, {scheduler}, LR={lr}")

            try:
                # Papermill call
                pm.execute_notebook(
                    input_notebook,  # Input notebook
                    output_notebook,  # Output notebook
                    parameters={
                        "optimizer_type": optimizer,
                        "scheduler_type": scheduler,
                        "learning_rate": lr,  # Set LR
                        "postfix": f"_independentdata_seed{seed}_{optimizer}_{scheduler}",
                        "train_dir": train_dir,
                        "validation_dir": validation_dir,
                        "epoch_num": epoch_num,
                        "random_seed": seed,  # Pass the seed parameter to the notebook
                    }
                )
                result = f"✅ SUCCESS: Seed: {seed}, Optimizer: {optimizer}, Scheduler: {scheduler}, LR: {lr}, Notebook: {output_notebook} completed successfully."

            except Exception as e:
                result = f"❌ ERROR in Seed: {seed}, Optimizer: {optimizer}, Scheduler: {scheduler}: {str(e)}"

            # Record result
            results.append((seed, optimizer, scheduler, lr, output_notebook))

            # Save log
            print(result)
            with open(log_file, "a") as log:
                log.write(result + "\n")

print("✅ All optimizer, scheduler, and seed experiments completed!")
